# Resistor inductor capacitor circuit model

Model of [RLC cicuit](https://en.wikipedia.org/wiki/RLC_circuit), by using the Euler's function
$e^{(a+ib)x}$ as solution for differential equation:
$$L\frac{\partial^2 f(t)}{\partial t^2} + R\frac{\partial f(t)}{\partial t} + \frac{1}C f(t) = 0$$

> Note:  
> Precisely, the $e^{(a+ib)x+(c+id)}$ is also a solution, where the exponent is a first degree polynomial
> with complex coefficients, like $\Omega x+\Phi$.
> However, a higher degree polynomial in the exponent should not be a solution, as the differential equation
> will require specific values of $t$.

The exponent coefficients can be easily calculated from a few circuit signal values over the time.
The problem is that this requires the _actual complex_ values of this asignal, but in real-world scenario
we have the _real_ parts only.

This is an attempt to reconstruct the _actual imaginary_ part of a circuit signal values, in order to
calculate all the coefficients of the Euler's function, that drives that signal.

In [ ]:
# Preparation
import cmath
import matplotlib.pyplot as plt

def calc_euler(euler_omega: complex, t: float) -> complex:
    """Euler foumula"""
    return cmath.e ** (euler_omega * t)

# Input values
OMEGA = complex(-cmath.pi/32, cmath.pi/6)
t_range = range(40)
func_vals = [calc_euler(OMEGA, t) for t in t_range]

## Reference plots

These plots demonstrate the simplification, to be achieved by using the actual complex signal value.
Of course, this requires to discover its imaginary component part, by using the real one.

### Complex logarithm

Below is a plot of the real part of the base function $e^{(a+ib)x}$ and its logarithm.

> Both real and imaginary components of the logarithm are linear, even if the imaginary wraps around $\pm\pi$.
> However, this is just because this is a logarithm from the actual complex value.

In [ ]:
# Function and logarithm plot
log_vals = [cmath.log(v) for v in func_vals]

plt.plot([v.real for v in func_vals], label='function')
plt.plot([v.real for v in log_vals], label='logarithm/real')
plt.plot([v.imag for v in log_vals], label='logarithm/real')
plt.legend()
plt.show()

### Complex ratio with the derivative

Below is a plot of the real part of the base function $e^{(a+ib)x}$ and its derivative $(a+ib)e^{(a+ib)x}$,
together with their ratio $\frac{(a+ib)e^{(a+ib)x}}{e^{(a+ib)x}} = a+ib$.

> Both real and imaginary components of the ratio are constant.
> However, this is just because we are dividing the actual complex values.

In [ ]:
# Function and derivative plot
deriv_vals = [(OMEGA * calc_euler(OMEGA, t)) for t in t_range]

plt.plot([v.real for v in func_vals], label='function')
plt.plot([v.real for v in deriv_vals], label='derivative')
plt.plot([(dv/fv).real for fv,dv in zip(func_vals, deriv_vals)],
        label='ratio/real', linestyle='--')
plt.plot([(dv/fv).imag for fv,dv in zip(func_vals, deriv_vals)],
        label='ratio/imag', linestyle='--')
plt.legend()
plt.show()

## Deduction of signal imaginary value

### Using real part of derivative and function only

Since $e^{(a+ib)x} = e^{ax}cos(bx) + i.e^{ax}sin(bx)$, the derivative of its real part
$e^{ax}cos(bx)$ is:

$$\frac{\partial e^{ax}cos(bx)}{\partial x} = ae^{ax}cos(bx) - be^{ax}sin(bx)$$

> The real part of actual derivative is the same as the derivative of the real part above.

#### Main problem to be solved

The ratio between these real parts is **NOT** a constant, but instead:
$$\frac{ae^{ax}cos(bx) - be^{ax}sin(bx)}{e^{ax}cos(bx)} = a - b.tan(bx)$$

In [ ]:
# Ratio between real part of derivative and function plot
EPSILON=1e-14
def crop_val(v):
    return v if abs(v) < 1/EPSILON else cmath.nan
plt.plot([v.real for v in func_vals], label='function')
plt.plot([v.real for v in deriv_vals], label='derivative')
plt.plot([crop_val(dv.real/fv.real) for fv,dv in zip(func_vals, deriv_vals)],
        label='ratio')
plt.plot([OMEGA.real - OMEGA.imag*crop_val(cmath.tan(OMEGA.imag*t)) for t in t_range],
        label='a-b.tan(b)', linestyle=':')
plt.legend()
plt.show()